<h1>Introduction</h1>
Toronto is both an incredibly beautiful and diverse city to live in. Canadians are also avid coffee drinkers. As a potential Toronto migrant and coffee lover, I want to know which Toronto  neighbourhood has the most independent coffee shops or cafes to indulge my coffee cravings. We will explore both raw numbers and also customer reviews of these spots.

<h1>Data</h1>
We're going to use the FourSquare API and map visuals to aggregate data and plot coffee shop locations to display coffee shop concentrations in both cities by neighbourhood. We will use the Wikipedia page provided in an earlier lab to compile Toronto and New York neighbourhood names to input to the FourSquare API.